In [ ]:
!pip install langchain
!pip install PyPDF2
!pip install qdrant_client
!pip install langchain_openai
!pip install pypdf
!pip install openai

In [2]:
import os

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Qdrant
from langchain.chains import ConversationalRetrievalChain
from qdrant_client import QdrantClient,models
from qdrant_client.http.models import PointStruct
from openai import OpenAI

In [3]:
os.environ["OPENAI_API_KEY"]=""
os.environ["QDRANT_URL"]=""
os.environ["QDRANT_COLLECTION_NAME"]="health_report"

In [ ]:
#create new cluseter in qdrant
record=0

connection = QdrantClient(
    url=os.environ.get("QDRANT_URL"),
    api_key="",
)



info = connection.get_collection(collection_name=os.environ.get("QDRANT_COLLECTION_NAME"))

print("Collection info:", info)
for get_info in info:
  print(get_info)

In [5]:
def get_embeddings():
  return OpenAIEmbeddings(
    openai_api_key=os.environ.get("OPENAI_API_KEY"),
    chunk_size=1
  )

In [6]:
from langchain_openai import ChatOpenAI
def get_chat_model():
  return ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature=0)

In [7]:
def ask_question_with_context(qa, question, chat_history):
    chat_history = []
    query = ""
    result = qa({"question": question, "chat_history": chat_history})
    print("answer:", result["answer"])
    chat_history = [(query, result["answer"])]
    return chat_history

In [8]:
# from langchain.prompts import PromptTemplate
from langchain_core.prompts import SystemMessagePromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate
def main():

    embeddings = get_embeddings()

    llm = get_chat_model()

    vector_store = Qdrant(
        client=connection,
        collection_name=os.getenv("QDRANT_COLLECTION_NAME"),
        embeddings=embeddings,
    )
    system_template = """你是只會用繁體中文的護士，只能回答健康檢查、三高有關的問題，其他問題一律不回答。必須只用繁體中文回答問題。
        ----------------
        {context}"""
    messages = [
        SystemMessagePromptTemplate.from_template(system_template),
        HumanMessagePromptTemplate.from_template("{question}")
        ]
    qa_prompt = ChatPromptTemplate.from_messages(messages)

    qa = ConversationalRetrievalChain.from_llm(
          llm=llm,
          retriever=vector_store.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.7, "k": 3}),
          return_source_documents=True,
          condense_question_llm = llm,
          combine_docs_chain_kwargs={"prompt": qa_prompt})

    chat_history = []
    while True:
        query = input('you: ')
        if query == 'q':
            break
        chat_history = ask_question_with_context(qa, query, chat_history)


if __name__ == "__main__":
    main()


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


you: 糖尿病患血糖控管不佳會造成那些併發症?


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


answer: 糖尿病患血糖控制不佳可能會造成以下併發症：
1. 心臟、血管病變：包括高血壓、心絞痛、心肌梗塞、腦中風等。
2. 全身感染：例如泌尿道感染、肺結核、黴菌感染、香港腳、體癬、灰指甲、足癬、帶狀皰疹等。
3. 神經病變：包括手腳酸麻、肌肉變形、足部變形等。
這些併發症可能會導致嚴重的健康問題，因此重要控制血糖，以預防這些併發症的發生。
you: 總膽固醇200mg/dl, 三酸甘油酯110mg/dl, 這樣算高血脂症嗎?
answer: 根據衛教資料，總膽固醇理想濃度為＜200mg/dl，而三酸甘油酯的理想濃度為＜150mg/dl。根據你提供的數值，總膽固醇200mg/dl已達邊緣值，而三酸甘油酯110mg/dl在正常範圍內。因此，根據這些數據，你的總膽固醇已達邊緣值，建議密切注意飲食及生活習慣，以預防高血脂症的發展。
you: 腦中風跟高血脂有關係嗎？
answer: 腦中風與高血脂有關係。高血脂症會導致動脈粥樣硬化，也就是動脈壁上的脂肪堆積，進而造成血管阻塞或破裂，增加腦中風的風險。因此，控制高血脂可以降低罹患腦中風的機率。
you: 健檢項目越多越好嗎？
answer: 健檢項目的選擇應該是因人而異的，並不是越多越好。選擇適合自己的健檢項目才是最重要的。根據個人的年齡、性別、家族病史、生活環境、生活習慣等因素，尋找適合自己的健檢項目才能達到最佳效果。建議向專業的醫療團隊諮詢，找出符合自己需求的健檢項目。
you: 如果我不控制三高，會發生什麼？
answer: 如果您不控制三高（高血壓、高血脂、高血糖），可能會引發以下慢性併發症：

1. 大血管病變：包括腦中風、心肌梗塞、週邊動脈狹窄等。
2. 小血管病變：可能導致視網膜病變、腎臟病變等。
3. 神經病變：可能出現手腳酸麻、肌肉變形、足部變形等症狀。

因此，控制三高是非常重要的，可以透過飲食、運動、藥物等方式改善症狀，降低合併症的風險。如果有任何疑問或需要更詳細的建議，建議您咨詢醫師進行評估和治療。
you: 患有三高的人應該遵循什麼飲食原則?
answer: 患有三高的人應該遵循以下飲食原則：
1. 採六大均衡飲食定時定量，選擇少油、少鹽、不加糖、不勾芡、多纖維食物。
2. 控制油量攝取，少吃油炸、油煎或油酥的食物；少吃豬皮、雞皮、鴨皮、魚皮等。
3. 炒菜宜選用不飽和脂肪酸高的油脂，如橄欖油